In [11]:
import numpy as np
import math
import pygame
import threading
import time
import cv2
import torch
import sys

np.set_printoptions(threshold=sys.maxsize)

In [12]:
class DecisionalNN(torch.nn.Module):
    def __init__(self, _input_size, _output_size, _device='cpu'):
        super().__init__()

        self.conv1 = torch.nn.Conv2d(_input_size, 128, 3)
        self.pool1 = torch.nn.MaxPool2d(3)
        self.conv2 = torch.nn.Conv2d(128, 256, 3)
        self.pool2 = torch.nn.MaxPool2d(3)

        self.dense1 = torch.nn.Linear(256, 128)
        self.dense2 = torch.nn.Linear(128, _output_size)

        self.to(_device)

    def forward(self, _x):
        #print (f'init x: {_x.shape}')
        x = self.conv1(_x)
        #print (f'conv1 x: {x.shape}')
        x = self.pool1(x)
        #print (f'pool1 x: {x.shape}')
        x = self.conv2(x)
        #print (f'conv2 x: {x.shape}')
        x = self.pool2(x)
        #print (f'pool2 x: {x.shape}')

        x = torch.flatten(x)
        #print (f'flat x: {x.shape}')
        x = self.dense1(x)
        #print (f'dense1 x: {x.shape}')
        x = torch.relu(x)
        x = self.dense2(x)
        #print (f'dense2 x: {x.shape}')
        x = torch.relu(x)

        # * normalize
        # x -= x.min() 
        # x /= x.max()
        # n = torch.norm(x)
        # if n == 0.0:
        #     n = 1e-10
        # x /= n

        return x

In [13]:
print (f'{math.dist(np.array([0.0, 0.5-(1/(4*np.sqrt(3))), 0.0]),  np.array([0.25, -1/(4*np.sqrt(3)), 0.0]))}')

0.5590169943749475


In [14]:
class RadialCell:
    def __init__(self, _pos, _channels, _id):
        self.pos = _pos
        self.chunk = None
        self.channels = _channels
        self.id = _id
        
    def get_color(self):
        return self.channels[0:4]

    def set_channels(self, _scale, _channels):
        self.channels = _channels
        self.channels[0:4] = np.clip(self.channels[0:4], 0.0, 1.0)

    def move(self, _scale, _influence):
        dir_a = _influence - self.pos
        norm = np.linalg.norm(dir_a)
        if norm != 0:
            dir_a = dir_a/norm
        else:
            dir_a = np.array([0.0, 0.0, 0.0])
        self.pos = self.pos + (dir_a *_scale)

class GridSystem:
    def __init__(self, _size):
        self.size = _size
        self.chunks = {}

    def add_cell(self, _cell):
        c = self.pos_to_chunk(_cell.pos)
        _cell.chunk = c
        k = str(c)
        if k not in self.chunks:
            self.chunks[k] = []
        self.chunks[k].append(_cell)

    def remove_cell(self, _cell):
        c = _cell.chunk
        k = str(c)
        self.chunks[k].remove(_cell)

    def update_cell_chunk(self, _cell):
        c = _cell.chunk
        nc = self.pos_to_chunk(_cell.pos)
        if (c != nc).all():
            self.chunks[str(c)].remove(_cell)
            self.add_cell(_cell)

    def pos_to_chunk(self, _pos):
        return np.ceil(_pos / self.size).astype(int)

    def query_neighbors_in_radius(self, _cell):
        neighbors = []
        pos = _cell.pos
        chunk = _cell.chunk
        for x in range(int(chunk[0])-1, int(chunk[0])+2, 1):
            for y in range(int(chunk[1])-1, int(chunk[1])+2, 1):
                k = str(np.array([x, y, 0]).astype(int))
                if k in self.chunks:
                    for cell in self.chunks[k]:
                        if cell.id != _cell.id:
                            dis = math.dist(pos, cell.pos)
                            if dis < self.size:
                                neighbors.append(cell)
        return neighbors


class RadialAutomata:
    def __init__(self, _radius, _map_res=8, _rate=0.01, _color_scale=0.2, _move_scale=0.1, _cell_limit=2048, _neighbor_limit=32):
        self.radius = _radius
        self.map_res = _map_res
        self.rate = _rate
        self.id_count = 0
        self.color_scale = _color_scale
        self.move_scale = _move_scale
        self.cell_limit = _cell_limit
        self.neighbor_limit = _neighbor_limit
        self.chunk_size = _radius
        self.reset()

    def reset(self):
        self.move_nn = DecisionalNN(16, 2)
        self.color_nn = DecisionalNN(16, 16)
        self.grid = GridSystem(self.radius)
        self.cells = []

        # * init first cell(s)
        init_pos = np.array([0.0, 0.5-(1/(np.sqrt(3)*4)), 0.0])
        init_channels = np.zeros([16])
        init_channels[0:4] = np.array([0.0, 0.0, 1.0, 1.0])
        init_cell = RadialCell(init_pos, init_channels, self.id_count)
        self.grid.add_cell(init_cell)
        self.cells.append(init_cell)
        self.id_count += 1

        init_pos = np.array([0.25, -(1/(np.sqrt(3)*4)), 0.0])
        init_channels = np.zeros([16])
        init_channels[0:4] = np.array([1.0, 0.0, 0.0, 1.0])
        init_cell = RadialCell(init_pos, init_channels, self.id_count)
        self.grid.add_cell(init_cell)
        self.cells.append(init_cell)
        self.id_count += 1
        
        init_pos = np.array([-0.25, -(1/(np.sqrt(3)*4)), 0.0])
        init_channels = np.zeros([16])
        init_channels[0:4] = np.array([0.0, 1.0, 0.0, 1.0])
        init_cell = RadialCell(init_pos, init_channels, self.id_count)
        self.grid.add_cell(init_cell)
        self.cells.append(init_cell)
        self.id_count += 1

    def pixelize(self, _scale, _size):
        image = np.ones([_size, _size, 3]).astype(np.float32)
        num_cells = {}
        
        # * find center-most cell and use as origin
        cen = np.array([0.0, 0.0, 0.0])
        for cell in self.cells:
            cen += cell.pos
        cen /= len(self.cells)
        cen[2] = 0.0
        
        for cell in self.cells:
            cell_pos = cell.pos.copy()
            pos = ((cell_pos - cen) * _scale) + (_size/2, _size/2, 0.0)
            pos = pos.astype(int)[0:2]
            if pos[0] < _size and pos[0] >= 0 and pos[1] < _size and pos[1] >= 0:
                color = cell.get_color()
                rgb, a = color[0:3], color[3]
                color = np.clip(1.0 - a + rgb, 0.0, 1.0)
                # * blend color if pixel already colored
                key = str(pos)
                if key in num_cells:
                    image[pos[0], pos[1]] *= num_cells[key]
                    image[pos[0], pos[1]] += color[::-1]
                    num_cells[key] += 1
                    image[pos[0], pos[1]] /= num_cells[key]
                else:
                    num_cells[key] = 1
                    image[pos[0], pos[1]] = color[::-1]
        image = np.array(image * 255.0).astype(np.uint8)
        return image

    def update(self):
        # * stochastic update
        stochastic_mask = np.random.rand(len(self.cells)) < self.rate
        active_cells =  np.argwhere(stochastic_mask).flatten()
        
        # * perception step
        perception, neighbors = self.percieve(active_cells)
        
        # * selection step
        actions = self.select(perception)
        
        # * each active cell performs its action
        self.perform(active_cells, actions, perception, neighbors)
        
    def percieve(self, _active_cells):
        perception = torch.zeros([len(_active_cells), 16, self.map_res*2+1, self.map_res*2+1])
        neighbors_count = []

        for i, c in enumerate(_active_cells):
            cell = self.cells[c]

            # * init perception map
            perception_map = np.zeros([self.map_res*2+1, self.map_res*2+1, 16])
            perception_map[self.map_res, self.map_res] = cell.channels

            # currate cells from current and adjacent chunks
            neighbors = self.grid.query_neighbors_in_radius(cell)
            for neighbor in neighbors:
                dir = neighbor.pos - cell.pos
                dir = dir / self.radius
                loc = np.floor(dir * self.map_res).astype(int)
                loc += self.map_res
                perception_map[loc[0], loc[1]] = neighbor.channels

            # * add to overall perception
            x = torch.tensor(perception_map, dtype=torch.float32).permute(2, 0, 1)
            perception[i] = x
            neighbors_count.append(len(neighbors))

            # * show perception map
            # image = (perception_map*255).astype(np.uint8)
            # image = np.rot90(image, 1)
            # scale = 16
            # cv2.imshow('perception map', cv2.resize(image, (17*scale, 17*scale), interpolation=cv2.INTER_NEAREST))
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

        return perception, neighbors_count
    
    def select(self, _perception):
        # * types of actions:
        #       - [0] dupicate
        #       - [1] destruct
        #       - [2] change color
        #       - [3] move
        actions = [] 
        for i, _ in enumerate(_perception):
            # * for now, select action at random
            # * TODO train nn to take perception and select action (?)
            n = np.random.randint(0, 100)
            if n < 20:
                action = 0
            elif n >= 20 and n < 60:
                action = 2
            else:
                action = 3
            actions.append(action)
        return actions
    
    def perform(self, _cells, _actions, _perception, _neighbors):
        # * perform non-size-changing actions:
        for i in range(len(_cells)):
            cell = _cells[i]
            action = _actions[i]
            perception = _perception[i]

            # * change a cell's color slightly
            if action == 2:
                res = self.color_nn(perception).detach().numpy()
                x =  np.clip(res, 0.0, 1.0)
                print (f'color_nn res: \n{x}')
                self.cells[cell].set_channels(self.color_scale, x)

            # * move a cell's position slightly
            elif action == 3:
                curr_cell = self.cells[cell]
                res = self.move_nn(perception).detach().numpy()
                print (f'move: {res}')
                x = np.clip(res, 0.0, 1.0)*2.0-1.0
                x = np.array([x[0], x[1], 0.0])
                print (f'move_nn res: \n{x}')
                curr_cell.move(self.move_scale, x)
                self.grid.update_cell_chunk(curr_cell)
                
        # * perform size-changing actions:
        new_cells = self.cells.copy()
        for i in range(len(_cells)):
            cell = _cells[i]
            action = _actions[i]
            perception = _perception[i]
            curr_cell = self.cells[cell]

            # * duplicate cell in random direction
            if action == 0 and len(new_cells) < self.cell_limit and _neighbors[i] <= self.neighbor_limit:
                print ('duplicate!')
                dup_pos = curr_cell.pos + ((np.random.rand(3) * 2.0) - 1) * self.move_scale
                dup_pos[2] = 0.0
                new_cell = RadialCell(dup_pos, curr_cell.channels, self.id_count)
                self.grid.add_cell(new_cell)
                new_cells.append(new_cell)
                self.id_count += 1
                
            # * destroy cell (if action dictates or over-populated)
            elif action == 1 or _neighbors[i] > self.neighbor_limit or self.cells[cell].get_color()[3] < 0.1 or _neighbors[i] == 0:
                print ('delete!')
                old_cell = self.cells[cell]
                new_cells.remove(old_cell)
                self.grid.remove_cell(old_cell)

        self.cells = new_cells

In [15]:
_WINDOW_BG_COLOR_ = (255, 255, 255)
_WINDOW_TEXT_COLOR_ = (0, 0, 0)
_SIZE_ = 512
_SCALE_ = 64
_AUTO_RUN_ = False

_RADIUS_ = 1.0
_RATE_ = 1.0

pygame.init()
pygame.display.set_caption('radial automata simulation')

window = pygame.display.set_mode((_SIZE_, _SIZE_))
automata = RadialAutomata(_RADIUS_, _rate=_RATE_)

# * create clock for fps
clock = pygame.time.Clock()
delta_time = 0

# * setup text rendering
font_size = 20
my_font = pygame.font.SysFont('consolas', font_size)
fps_surface = my_font.render(f'{clock.get_fps() :.0f}', False, _WINDOW_TEXT_COLOR_)
cells_sureface = my_font.render(f'dots: {len(automata.cells)}', False, _WINDOW_TEXT_COLOR_)

# * mutex
mutex = threading.Lock()

running = True 
def run_sim(_delay):
    time.sleep(_delay)
    while running:
        mutex.acquire()
        automata.update()
        mutex.release()

# # * start forward worker
if _AUTO_RUN_:
    sim = threading.Thread(target=run_sim, args=[1], daemon=False)
    sim.start()

while running:
    # * close application
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            if event.key == pygame.K_RETURN:
                mutex.acquire()
                automata.update()
                mutex.release()
            if event.key == pygame.K_r:
                mutex.acquire()
                automata = RadialAutomata(_RADIUS_, _rate=_RATE_)
                mutex.release()
            if event.key == pygame.K_p:
                image = automata.pixelize(8, 32)
                image = np.rot90(image, 1)
                scale = 8
                cv2.imshow('perception map', cv2.resize(image, (32*scale, 32*scale), interpolation=cv2.INTER_NEAREST))
                cv2.waitKey(0)
                cv2.destroyAllWindows()
    
    # * draw tensor to window
    window.fill(_WINDOW_BG_COLOR_)
    
    # * blip cells
    cells = automata.cells

    for cell in cells:
        color = cell.get_color()
        rgb, a = color[:3], color[3:4]
        color = np.clip(1.0 - a + rgb, 0.0, 1.0)
        color = np.array(color * 255)
        color = np.clip(color, 0, 255).astype(int)
        cell_pos = cell.pos.copy()
        cell_pos[1] *= -1.0
        pos = (cell_pos*_SCALE_) + (_SIZE_/2, _SIZE_/2, 0.0)
        pygame.draw.circle(window, color, pos[:2,], 5, 5) #(r, g, b) is color, (x, y) is center, R is radius and w is the thickness of the circle border.
    
    # * calculate fps
    delta_time = clock.tick()
    fps_surface = my_font.render(f'{clock.get_fps() :.0f}', False, _WINDOW_TEXT_COLOR_)
    window.blit(fps_surface, (0, 0))

    # * show number of cells
    cells_sureface = my_font.render(f'dots: {len(cells)}', False, _WINDOW_TEXT_COLOR_)
    window.blit(cells_sureface, (0, _SIZE_-font_size))
    
    # * flip it!
    pygame.display.flip()

# * quit it!
pygame.quit()

if _AUTO_RUN_:
    sim.join()

move: [1. 0.]
move_nn res: 
[ 1. -1.  0.]
color_nn res: 
[0.         0.         0.         0.791004   0.         0.14753991
 0.         0.10999402 0.         0.         0.3868902  0.
 0.27298343 0.         0.         0.34094283]
duplicate!
